# Introduction to Brain Segmentation with Keras

# ***MAIN 2019 Educational Course***

## Thomas Funck

## McGill University

## **Contact**: email: [tffunck@gmail.com](mailto:tffunck@gmail.com) , Twitter: [@tffunck](https://twitter.com/tffunck)

In [ ]:
#Download repository
!git clone https://github.com/tfunck/minc_keras

In [ ]:
#Switch dir
import os
if os.getcwd() != '/content/minc_keras/' : os.chdir('/content/minc_keras')

#Unzip
!tar -jxvf data/output.tar.bz2 &> /dev/null

In [ ]:
#Import minc_keras
import minc_keras

# ***Outline***
# 1. Basic Concepts
## --1.1 Introductory Examples
## --1.2 Parameters
## --1.3 Receptive Field
## --1.4 Dilations
## --1.5 Cross-validation
## --1.6 Final activation functions
## --1.7 Cross-entropy
# 2. Simple Networks
## --2.1 Base
## --2.2 Over-fitting
## --2.3 Regularization with Drop-out
## --2.4 Using dilations
# 3. U-Net
## --3.1 Max-pool 
## --3.2 Upsampling
## --3.3 Concatenating

# Example of a simple convolutional neural network
## 1 Layer, 3 Kernels of 3x3 size

![](https://github.com/tfunck/minc_keras/blob/master/images/conv_1.png?raw=1)

## ***Keras Output:***

In [ ]:
minc_keras.minc_keras(source_dir="output", target_dir="results", input_str="T1w_anat_rsl.mnc",\
           label_str="variant-seg_rsl.mnc", ratios=[0.7,0.15,0.15], nK="3", kernel_size=3,\
           model_type="custom", nb_epoch=0, make_model_only=True, verbose=0) 

```
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 110, 92, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 110, 92, 3)        30        
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 92, 3)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 110, 92, 3)        12        
=================================================================
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________
None
```

# Adding in a second layer
## 2 Layers, 3 Kernels of 3x3 size
![](https://github.com/tfunck/minc_keras/blob/master/images/conv_2.png?raw=1)
## ***Keras Output:***

In [ ]:
minc_keras.minc_keras(source_dir="data/output", target_dir="results", input_str="T1w_anat_rsl.mnc",\
           label_str="variant-seg_rsl.mnc", ratios=[0.7,0.15,0.15], nK="3,3", kernel_size=3,\
           model_type="custom", nb_epoch=0, make_model_only=True)

# Adding in a second layer
## 3 Layers, 3 Kernels of 3x3 size
![](https://github.com/tfunck/minc_keras/blob/master/images/conv_3.png?raw=1)
## ***Keras Output:***

In [ ]:
minc_keras.minc_keras(source_dir="data/output", target_dir="results", input_str="T1w_anat_rsl.mnc",\
           label_str="variant-seg_rsl.mnc", ratios=[0.7,0.15,0.15], nK="3,3,3", kernel_size=3,\
           model_type="custom", nb_epoch=0, make_model_only=True)

# How many parameters in a convolutional layer?
## $N_{parameters} = N_{kernels} \times Kernel_{Dimension1} \times Kernel_{Dimension2}  + N_{kernels}$
### ***Example: Layer with 3 kernels of 3x3 size + 1 bias parameter per kernel = 3x3x3+3 = 30 parameters*** 


```
Layer (type)                 Output Shape              Param #   
=================================================================
input_2 (InputLayer)         (None, 110, 92, 1)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 110, 92, 3)        30        
_________________________________________________________________
dropout_2 (Dropout)          (None, 110, 92, 3)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 110, 92, 3)        84        
_________________________________________________________________
dropout_3 (Dropout)          (None, 110, 92, 3)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 110, 92, 3)        84        
_________________________________________________________________
dropout_4 (Dropout)          (None, 110, 92, 3)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 110, 92, 3)        12        
=================================================================
Total params: 210
Trainable params: 210
Non-trainable params: 0
_________________________________________________________________
None
```
# Parameters in ***conv2d_3***  and ***conv2d_4*** are different? What gives?

![](https://github.com/tfunck/minc_keras/blob/master/images/depth_0_1.png?raw=1)
## Layer with N kernels with turn a 2D (X,Y) input into a 3D (X,Y,N) array
## --> 3x3 kernels for next layer must have dimensions of (3,3,N) to convolve (X,Y,N) image

![](https://github.com/tfunck/minc_keras/blob/master/images/depth_0_2.png?raw=1)
## $N_{parameters} = N_{kernels} \times Kernel_{Dimension1} \times Kernel_{Dimension2} \times Input_{Channels}  + N_{kernels}$
### ***Example: Layer with 3 3x3 kernels and 3 channels + 3 bias parameters = 3x3x3x3+3 = 84 parameters*** 

# How to transform multi-channel layers so that final output layer matches labels? 

![](https://github.com/tfunck/minc_keras/blob/master/images/conv_1.png?raw=1)

![](https://github.com/tfunck/minc_keras/blob/master/images/depth_1.png?raw=1)

![](https://github.com/tfunck/minc_keras/blob/master/images/depth_2.png?raw=1)

# Receptive Field

## 3 Layers, 16 Kernels of 3x3 size
![](https://github.com/tfunck/minc_keras/blob/master/images/conv_3.png?raw=1)

## 
![](https://github.com/tfunck/minc_keras/blob/master/images/receptive_field_0.png?raw=1)

![](https://github.com/tfunck/minc_keras/blob/master/images/receptive_field_1.png?raw=1)

# Dilations

# __Spliting Data for Cross Validation__
## 1) Uncorrelated data
## ***Train*** : Data on which network will be trained
## ***Validation*** : Data on which network is evaluated between iterations
## ***Test*** : Data for final evaluation of network
![](https://github.com/tfunck/minc_keras/blob/master/images/splits_a_1.png?raw=0)

## 2) Correlated data

### Example: T1 MRI images collected from on different scanners
![](https://github.com/tfunck/minc_keras/blob/master/images/splits_b_1.png?raw=1)

### ***Don't*** create splits with only very unbalanced splits:
![](https://github.com/tfunck/minc_keras/blob/master/images/splits_b_2.png?raw=1)
### ***Do*** balance your subtypes between splits:
![](https://github.com/tfunck/minc_keras/blob/master/images/splits_b_3.png?raw=1)

## 3) Splits with correlated data and repeated subjects
### 3 images x 5 subjects
### 3 subtypes of images (e.g. scanner type)
![](https://github.com/tfunck/minc_keras/blob/master/images/splits_c.png?raw=1)

# Training a simple model

In [ ]:
minc_keras.minc_keras(source_dir="data/output", target_dir="results", input_str="T1w_anat_rsl.mnc",\
           label_str="variant-seg_rsl.mnc", ratios=[0.7,0.15,0.15], nK="16,16,16", kernel_size=3,\
           model_type="custom", nb_epoch=5)

## Understanding Keras output
```
Layer (type)                 Output Shape              Param #   
=================================================================
input_7 (InputLayer)         (None, 110, 92, 1)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 110, 92, 16)       160       
_________________________________________________________________
dropout_10 (Dropout)         (None, 110, 92, 16)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 110, 92, 16)       2320      
_________________________________________________________________
dropout_11 (Dropout)         (None, 110, 92, 16)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 110, 92, 16)       2320      
_________________________________________________________________
dropout_12 (Dropout)         (None, 110, 92, 16)       0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 110, 92, 3)        51        
=================================================================
Total params: 4,851
Trainable params: 4,851
Non-trainable params: 0
_________________________________________________________________
None
Model: results/model/model.hdf5
Running with 5
(11598, 110, 92, 1) (11598, 110, 92, 3) (2556, 110, 92, 1) (2556, 110, 92, 3)
Train on 11598 samples, validate on 2556 samples
Epoch 1/5
11598/11598 [==============================] - 355s 31ms/step - loss: 0.7399 - categorical_accuracy: 0.8595 - val_loss: 0.2688 - val_categorical_accuracy: 0.8872
Epoch 2/5
11598/11598 [==============================] - 353s 30ms/step - loss: 0.2238 - categorical_accuracy: 0.9010 - val_loss: 0.2193 - val_categorical_accuracy: 0.8986
Epoch 3/5
11598/11598 [==============================] - 352s 30ms/step - loss: 0.1969 - categorical_accuracy: 0.9111 - val_loss: 0.1999 - val_categorical_accuracy: 0.9083
Epoch 4/5
11598/11598 [==============================] - 350s 30ms/step - loss: 0.1807 - categorical_accuracy: 0.9204 - val_loss: 0.1863 - val_categorical_accuracy: 0.9184
Epoch 5/5
11598/11598 [==============================] - 352s 30ms/step - loss: 0.1689 - categorical_accuracy: 0.9272 - val_loss: 0.1745 - val_categorical_accuracy: 0.9243
2478/2478 [==============================] - 21s 9ms/step
Test: Loss= 0.16739090936765064 Metric= 0.9286423676239087
No images were specified for prediction.
Model training plot written to  results/report//model_loss_plot.png
```

# Identifying over-fitting

In [2]:
import minc_keras
minc_keras.minc_keras(source_dir="data/output", target_dir="results", input_str="T1w_anat_rsl.mnc",model_fn="overfit.hdf5",\
           label_str="variant-seg_rsl.mnc", ratios=[0.3,0.35,0.35], nK="128,128,128,128,128", kernel_size=5,\
           model_type="custom", nb_epoch=3)

1 [92, 110, 92]
2 [92, 110, 92]
N Labels: 3
Drop out: 0
Number of Dilations: None
Activation hidden: relu
Activation output: sigmoid
Layer: 0 128 5 1
Layer: 1 128 5 1
Layer: 2 128 5 1
Layer: 3 128 5 1
Layer: 4 128 5 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 110, 92, 1)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 110, 92, 128)      3328      
_________________________________________________________________
dropout_6 (Dropout)          (None, 110, 92, 128)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 110, 92, 128)      409728    
_________________________________________________________________
dropout_7 (Dropout)          (None, 110, 92, 128)      0         
________________________________________________________

KeyboardInterrupt: 

# Using regularization to control over-fitting

In [ ]:
minc_keras.minc_keras(source_dir="output/", target_dir="results", input_str="T1w_anat_rsl.mnc",\
           label_str="variant-seg_rsl.mnc", ratios=[0.7,0.15,0.15], nK="16,16,16", kernel_size=3,\
           model_type="custom", drop_out=0.25, nb_epoch=5)

# Adding dilations

In [ ]:
minc_keras.minc_keras(source_dir="output", target_dir="results", input_str="T1w_anat_rsl.mnc",model_fn="drop_out.hdf5",\
                      label_str="variant-seg_rsl.mnc", ratios=[0.7,0.15,0.15], nK="16,16,16", n_dil="2,2,2",\
                      kernel_size=3, model_type="custom", drop_out=0.25, nb_epoch=5)

## Configuring basic options

In [ ]:
from utils import *
import numpy as np
from minc_keras import *
### Set input and label string
input_str='pet.mnc' 
label_str='dtissue.mnc'

### Set filename for .csv that will store data frame 
images_fn='data.csv'

### Set source directory from which data will be read
source_dir="/data1/users/tfunck/pet/data_ses/"

### Set the target directory where output results will be saved
target_dir="/data1/users/tfunck/pet/results"

### Set raiots for train/validation/test
ratios=[0.7,0.15]

### By default we set clobber to False so that we don't overwrite existing files
### Feel free to change if needed
clobber=True

### Size of batches that will be passed to model. The default 2 makes it easy
batch_size=2

### Image dimensions. We are slicing the 3D images into 2D slices. This serves to augment the data
### and make training faster
image_dim=2

setup_dirs()  

### Set filename for .csv file that will contain info about input images
images_fn = set_model_name(images_fn, report_dir, '.csv')

![](https://github.com/tfunck/minc_keras/blob/master/images/splits_c_3.png?raw=1)

In [ ]:
[images, data] = prepare_data(source_dir, data_dir, report_dir, input_str, label_str, ratios, batch_size,feature_dim, images_fn,  clobber=clobber)

## Building a U-NET in Keras

![](https://github.com/tfunck/minc_keras/blob/master/images/unet.png?raw=1)

Ronneberger, Fischer, and Brox. 2015."U-net: Convolutional networks for biomedical image segmentation." International Conference on Medical image computing and computer-assisted intervention. https://arxiv.org/abs/1505.04597

In [ ]:
    ### 1) Define architecture of neural network
    Y_validate=np.load(data["validate_y_fn"]+'.npy')
    nlabels=len(np.unique(Y_validate))#Number of unique labels in the labeled images
    
    img_rows=image_dim[1]
    img_cols=image_dim[2]
    nMLP=16
    nRshp=int(sqrt(nMLP))
    nUpSm=int(image_dim[0]/nRshp)
    image = Input(shape=(image_dim[1], image_dim[2],1))
    
    BN1 = BatchNormalization()(image)

In [ ]:
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(BN1)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

# Max Pool

![](https://github.com/tfunck/minc_keras/blob/master/images/max_pool.png?raw=1)

## Becareful: 2x2 max pool reduces image dimensions by 2
## --> Applying max pool 4 times reduces image dimensions by 2^4
## --> Make sure image can be evenly divided, otherwise end up with problems

In [ ]:
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

In [ ]:
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

In [ ]:
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

In [ ]:
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

In [ ]:
    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)
    conv6 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

# ***Upsampling***

# 1. Transpose Convolution

## $M \circledast k = N$, where $k$ is a $(x,x)$ max pool filter, $M$ is a $(y,y)$ 2D array
## $N$ is a downsampled 2D array with dimensions $(i,i)$, where $i = y-x+1$
![](https://github.com/tfunck/minc_keras/blob/master/images/upsample_1.png?raw=1)



## The 2D array can be "flattened" into a $(i^2,1)$ vector 1D $m$ 
## The convolution kernel, $k$, can also be expressed as a $(i^2,y^2)$ matrix $K$  
## --> $K m = n$, where $(i^2,y^2) \times (y^2,1) = (i^2,1)$ and $n$ is a $(y^2,1)$ 1D vector 

![](https://github.com/tfunck/minc_keras/blob/master/images/upsample_2.png?raw=1)

### If we take the transpose of the matrix $K$, $K^T$ to create a $(y^2,i^2)$ matrix
### We can write $K^T N = M_1$,  $(y^2,i^2) \times (i^2,1) = (y^2,1)$ which can then be converted back into a $(y,y)$ 2D array
![](https://github.com/tfunck/minc_keras/blob/master/images/upsample_3.png?raw=1)

# 2. Interpolation

## Simply use interpolation (e.g., nearest neighbour) to higher resolution
## Does not require extra parameters

# Merge

In [ ]:
    conv6_up = UpSampling2D(size=(2, 2))(conv6)
    conv6_pad = ZeroPadding2D( ((1,0),(1,0)) )(conv6_up)
    up7 = merge([conv6_pad, conv3], mode='concat', concat_axis=3)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

In [ ]:
    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

In [ ]:
    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=3)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

# ***Choosing final activation function***
## Last activation function is used to compare output of network to labels
## Type of activation function depends on purpose of network
## Classification often means mapping real valued inputs to integer outputs
## --> Shouldn't use ReLu for classification 


![](https://github.com/tfunck/minc_keras/blob/master/images/pet_mask.png?raw=1)

## Binary classification
![](https://github.com/tfunck/minc_keras/blob/master/images/sigmoid.png?raw=1)
![](https://github.com/tfunck/minc_keras/blob/master/images/sigmoid_eq.png?raw=1)

## Multi-category classification
### Generalization of sigmoid function
### Creates a pseudo-probability distribution for each label: values between 0 and 1 and sum to 1
### Label with highest probability is attributed to pixel 

![](https://github.com/tfunck/minc_keras/blob/master/images/multi_sigmoid_eq.png?raw=1)

In [ ]:
### Output activation function
activation_output="softmax"

In [ ]:
    conv10 = Convolution2D(nlabels, 1, 1, activation=activation)(conv9)

    model = keras.models.Model(input=[image], output=conv10)

In [ ]:
### 2) Train network on data
model_fn =set_model_name(model_fn, model_dir)
history_fn = splitext(model_fn)[0] + '_history.json'

print( 'Model:', model_fn)
if not exists(model_fn) or clobber:
    #If model_fn does not exist, or user wishes to write over (clobber) existing model
    #then train a new model and save it
    
    #Load input images for training data
    X_train=np.load(data["train_x_fn"]+'.npy')
    #Load labels for training data
    Y_train=np.load(data["train_y_fn"]+'.npy')
    #Load input images for validation data set
    X_validate=np.load(data["validate_x_fn"]+'.npy')
    #Set compiler
    ada = keras.optimizers.Adam(0.0001)
    #Create filename to save checkpoints 
    checkpoint_fn = splitext(model_name)[0]+"_checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5"
    #Create checkpoint callback function for model
    checkpoint = ModelCheckpoint(checkpoint_fn, monitor='val_loss', verbose=0, save_best_only=True, mode='max')
    #Compile the model
    model.compile(loss = loss, optimizer=ada,metrics=[metric] )
    
  
    print("Running with", nb_epoch)
    #
    if loss in categorical_functions : 
        #Convert training data to categorical format
        Y_train = to_categorical(Y_train, num_classes=nlabels)
        #Convert validation data to categorical format
        Y_validate = to_categorical(Y_validate, num_classes=nlabels)
    #Fit model
    history = model.fit([X_train],Y_train,  validation_data=([X_validate], Y_validate), epochs = nb_epoch,callbacks=[ checkpoint])
    #save model   
    model.save(model_name)

    with open(history_fn, 'w+') as fp: json.dump(history.history, fp)
        
        

In [ ]:
    ### 3) Evaluate model on test data
    model = load_model(model_fn)
    X_test=np.load(data["test_x_fn"]+'.npy')
    Y_test=np.load(data["test_y_fn"]+'.npy')
    if loss in categorical_functions :
        Y_test=to_categorical(Y_test)
    test_score = model.evaluate(X_test, Y_test, verbose=1)
    print('Test: Loss=', test_score[0], 'Metric=', test_score[1])
    #np.savetxt(report_dir+os.sep+'model_evaluate.csv', np.array(test_score) )

    ### 4) Produce prediction
    #predict(model_fn, validate_dir, data_dir, images_fn, images_to_predict=images_to_predict, category="validate", verbose=verbose)
    #predict(model_fn, train_dir, data_dir, images_fn, images_to_predict=images_to_predict, category="train", verbose=verbose)
    predict(model_fn, test_dir, data_dir, images_fn, loss, images_to_predict=images_to_predict, category="test", verbose=verbose)
    plot_loss(metric, history_fn, model_fn, report_dir)